In [ ]:
#!pip install jcopdl
#!pip install gdown 

In [ ]:
# Download Datasets
!gdown https://drive.google.com/uc?id=12DT5Px7FQV7gEcyGWvKb5aZQW2ZptSP
!unzip /content/mnist.zip

In [1]:
import torch
from torch import nn, optim
from jcopdl.callback import Callback, set_config

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

## Datasets dan Dataloader (Hanya Trainset)

In [2]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [ ]:
bs = 64

data_transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.ToTensor()
])


train_set = datasets.ImageFolder("________", transform=data_transform)
trainloader = DataLoader(train_set, batch_size=bs, shuffle=True, num_workers=4)
